In [218]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

Autosaving every 300 seconds


# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [219]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36926,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.498289,5.96721,6,6,2017-08-13 23:50:48.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.999316,11.96720,12,12,2017-08-13 23:50:48.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,1.998630,23.93440,24,24,2017-08-13 23:50:48.0


#### NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
#### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

In [220]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)
ADNI2.head()


(6937, 94)


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,...,1.00000,1.25641,1.13549,NaN,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,...,1.33333,1.37838,1.50629,NaN,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


In [221]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)
ADNI2 = ADNI2.replace(-1, np.nan)
ADNI2 = ADNI2.replace(-4, np.nan)

In [222]:
# calculate the % of each column that is NaNs and print in descending order
nan_series = (ADNI2.isnull().sum()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(nan_series)

PIB                         1.000000
PIB_bl                      0.992936
FDG                         0.806833
AV45                        0.726251
FLDSTRENG                   0.553986
MidTemp                     0.543895
Fusiform                    0.543895
Entorhinal                  0.543895
Ventricles                  0.500793
Hippocampus                 0.497189
WholeBrain                  0.472106
ICV                         0.441401
FSVERSION                   0.441401
EcogSPOrgan                 0.363269
EcogPtOrgan                 0.356927
RAVLT_learning              0.354332
RAVLT_forgetting            0.353611
EcogSPDivatt                0.352314
MOCA                        0.350584
RAVLT_perc_forgetting       0.349575
EcogPtVisspat               0.347268
EcogPtDivatt                0.346836
EcogSPVisspat               0.346259
RAVLT_immediate             0.344385
EcogPtPlan                  0.342944
EcogSPPlan                  0.342655
EcogPtLang                  0.341790
A

In [223]:
ADNI2 = ADNI2.drop(['PIB','PIB_bl', 'FDG','AV45','FLDSTRENG','MidTemp','Fusiform','Entorhinal','Ventricles',
                    'Hippocampus','WholeBrain','ICV','FSVERSION'], axis=1)

#### Notes
- Our pure dataframe is called ADNI2

In [224]:
np.sum(ADNI2['VISCODE']=='bl')

789

In [225]:
encode = preprocessing.LabelEncoder()
ADNI2.loc[:, 'DX_bl'] = encode.fit_transform(ADNI2['DX_bl'])

In [226]:
ADNI2.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,4,66.9,Male,...,1.00000,1.00000,1.25641,1.13549,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,4,67.5,Male,...,1.25000,1.00000,1.25000,1.25195,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,4,67.5,Male,...,1.25000,1.00000,1.25000,1.25195,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,4,67.5,Male,...,1.25000,1.00000,1.25000,1.25195,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,4,68.5,Male,...,1.33333,1.33333,1.37838,1.50629,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


## Removing All Missing Data

In [227]:
ADNI2_remove_all_missing = copy(ADNI2[ADNI2['VISCODE']=='bl'])
for column in ADNI2_remove_all_missing.columns:
    if ADNI2_remove_all_missing[column].isnull().any():
        ADNI2_remove_all_missing = ADNI2_remove_all_missing.drop(labels=column, axis=1)

In [228]:
ADNI2_remove_all_missing.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,CDRSB,MMSE,EXAMDATE_bl,CDRSB_bl,MMSE_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,4,66.9,Male,...,0.0,29.0,2013-09-09,0.0,29,0.0,0.0,0,0,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,4,67.5,Male,...,0.0,30.0,2013-09-17,0.0,30,0.0,0.0,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,4,68.5,Male,...,0.0,30.0,2013-10-23,0.0,30,0.0,0.0,0,0,2017-10-06 23:20:00.0
142,5278,082_S_5278,bl,82,ADNI2,ADNI2,2013-09-04,4,80.2,Male,...,0.0,29.0,2013-09-04,0.0,29,0.0,0.0,0,0,2017-10-06 23:20:00.0
174,5277,027_S_5277,bl,27,ADNI2,ADNI2,2013-08-28,4,71.8,Female,...,0.5,28.0,2013-08-28,0.5,28,0.0,0.0,0,0,2017-11-14 12:46:35.0


In [229]:
ADNI2_remove_all_missing.shape

(789, 21)

In [230]:
ADNI2_remove_all_missing.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'CDRSB', 'MMSE', 'EXAMDATE_bl',
       'CDRSB_bl', 'MMSE_bl', 'Years_bl', 'Month_bl', 'Month', 'M',
       'update_stamp'],
      dtype='object')

In [231]:
categorical_predictors = ['PTGENDER']
continuous_predictors = ['RID', 'PTID', 'SITE', 'COLPROT', 'ORIGPROT', 'DX_bl', 'AGE', 
                         'EXAMDATE_bl', 'CDRSB_bl', 'MMSE_bl', 'PTEDUCAT']
dataframe_list = []
categorical_columns_g2cols = ['PTGENDER']                                 
#ADNI2_remove_all_missing_clean  = pd.get_dummies(ADNI2_remove_all_missing, columns=categorical_columns_g2cols, prefix = categorical_columns_g2cols , drop_first=False)
for i in range(len(categorical_predictors)):
    dummy_frame = pd.get_dummies(ADNI2_remove_all_missing[categorical_predictors[i]])
    # The commemted line below could be adjust, I just want to make the columns name clearer
    #dummy_frame = dummy_frame.iloc[:, :-1]
    dataframe_list.append(dummy_frame)
ADNI2_categorical = pd.concat(dataframe_list, axis=1)
ADNI2_remove_all_missing_clean = pd.concat([ADNI2_remove_all_missing[continuous_predictors], \
                                            ADNI2_categorical], axis=1)

In [232]:
print(ADNI2_remove_all_missing_clean.shape)
ADNI2_remove_all_missing_clean.head()


(789, 13)


,RID,PTID,SITE,COLPROT,ORIGPROT,DX_bl,AGE,EXAMDATE_bl,CDRSB_bl,MMSE_bl,PTEDUCAT,Female,Male
58,5282,082_S_5282,82,ADNI2,ADNI2,4,66.9,2013-09-09,0.0,29,17,0,1
91,5280,100_S_5280,100,ADNI2,ADNI2,4,67.5,2013-09-17,0.0,30,16,0,1
92,5279,082_S_5279,82,ADNI2,ADNI2,4,68.5,2013-10-23,0.0,30,20,0,1
142,5278,082_S_5278,82,ADNI2,ADNI2,4,80.2,2013-09-04,0.0,29,19,0,1
174,5277,027_S_5277,27,ADNI2,ADNI2,4,71.8,2013-08-28,0.5,28,15,1,0


In [233]:
ADNI2_remove_all_missing_clean.columns

Index(['RID', 'PTID', 'SITE', 'COLPROT', 'ORIGPROT', 'DX_bl', 'AGE',
       'EXAMDATE_bl', 'CDRSB_bl', 'MMSE_bl', 'PTEDUCAT', 'Female', 'Male'],
      dtype='object')

In [234]:
ADNI2_remove_all_missing_clean.to_csv(path_or_buf='ADNI2_remove_all_missing_v3.csv', index=False)

## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [235]:
for column in ADNI2.columns:
    print(column, ADNI2[column].unique())

RID [5282 5280 5279 ..., 5294 5295 5296]
PTID ['082_S_5282' '100_S_5280' '082_S_5279' ..., '051_S_5294' '057_S_5295'
 '053_S_5296']
VISCODE ['bl' 'm24' 'm06' 'm03' 'm36' 'm12' 'm48' 'm18' 'm30' 'm42' 'm54' 'm60'
 'm72' 'm66' 'm108' 'm96' 'm84' 'm78' 'm90' 'm120' 'm114' 'm102' 'm126']
SITE [ 82 100  27 135  53  36 153 127   7  32  18  33 130   2  41   9  16 126
  19  23  21 114  73  37  29  12  67   3  72  68  20  57 941  13   6 131
 128   5  52  24  70  51  22  99 141  11  31 123 116 137  94  35  14 109
 136  98  10 129]
COLPROT ['ADNI2']
ORIGPROT ['ADNI2' 'ADNIGO' 'ADNI1']
EXAMDATE ['2013-09-09' '2015-09-29' '2014-03-06' ..., '2017-08-16' '2016-08-22'
 '2015-09-30']
DX_bl [4 0 2 3 1]
AGE [ 66.9  67.5  68.5  80.2  71.8  78.   90.1  69.9  80.4  64.9  65.8  65.9
  75.1  75.2  73.5  78.1  78.8  77.9  70.4  68.1  56.5  66.1  73.9  74.3
  75.8  71.   67.2  87.8  62.7  80.3  85.3  68.   78.6  74.2  77.8  71.3
  75.7  82.5  69.6  65.4  78.2  85.8  76.1  68.7  74.9  85.2  59.3  76.8
  66.5  65

Entorhinal_bl [   nan  4946.  4065.  4313.  4043.  3375.  3772.  4315.  3146.  4766.
  4034.  3619.  4174.  3999.  3535.  2632.  4239.  5275.  3111.  3007.
  3568.  3671.  4206.  4138.  3592.  3143.  3082.  3586.  3537.  3388.
  4088.  1757.  4133.  4310.  3302.  3116.  2671.  2602.  3773.  2755.
  2523.  3066.  4527.  4587.  3292.  4220.  3304.  4163.  4162.  2928.
  4467.  3964.  3623.  3801.  3474.  5363.  2615.  4797.  3508.  3739.
  3587.  2382.  3863.  3615.  4706.  5019.  4736.  3842.  5116.  4046.
  3026.  2694.  4149.  3754.  3684.  4509.  4061.  4606.  3884.  3691.
  3714.  3303.  4036.  2588.  2103.  4353.  3299.  3812.  4203.  2725.
  4728.  3491.  3328.  3421.  3770.  3507.  3320.  2655.  2851.  3976.
  4146.  3994.  3309.  2766.  3820.  2473.  3857.  4170.  2461.  3381.
  4294.  3540.  3071.  2147.  2646.  4357.  3154.  3594.  3886.  2176.
  2910.  3229.  2518.  3289.  3900.  1883.  4436.  3599.  3025.  2170.
  2123.  2535.  3105.  3929.  3277.  2066.  2138.  3098.  4661.

In [236]:
ADNI2.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'DX', 'EXAMDATE_bl',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan

In [237]:
all_predictors = ['VISCODE','DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4',
                  'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate', 
                  'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 
                  'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 
                  'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt', 
                  'EcogSPTotal', 'FLDSTRENG', 'FSVERSION', 'Ventricles', 'Hippocampus', 'WholeBrain', 
                  'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl', 
                  'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 
                  'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 
                  'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                  'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl', 
                  'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 
                  'EcogSPPlan_bl', 'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'PIB_bl', 
                  'AV45_bl', 'Years_bl']
continuous_predictors = ['AGE'
                  ,'CDRSB_bl', 'ADAS11_bl', 
                  'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 
                  'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 
                  'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                  'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl', 
                  'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 
                  'EcogSPPlan_bl', 'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 
                  'AV45_bl', 'Years_bl', 'PTEDUCAT']

In [238]:
''' Pre-Processing the Missing Categorical Data '''
categorical_predictors = ['PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4']



''' Remove all categorical data with nan (only ~0.5%) '''
missing_rows = np.nonzero(np.sum(ADNI2[categorical_predictors].isnull(), axis=1))
ADNI2_categorical_clean = ADNI2.drop(ADNI2.index[missing_rows[0]])

''' Hot-One Encoding of Categorical Predictors '''
ADNI2_categorical = pd.get_dummies(ADNI2[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
    

In [239]:
ADNI2_categorical.head()

,PTGENDER_Female,PTGENDER_Male,PTETHCAT_Hisp/Latino,PTETHCAT_Not Hisp/Latino,PTRACCAT_Am Indian/Alaskan,PTRACCAT_Asian,PTRACCAT_Black,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0
58,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0
59,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0
68,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0
91,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0
92,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0


In [240]:
ADNI2_categorical.columns

Index(['PTGENDER_Female', 'PTGENDER_Male', 'PTETHCAT_Hisp/Latino',
       'PTETHCAT_Not Hisp/Latino', 'PTRACCAT_Am Indian/Alaskan',
       'PTRACCAT_Asian', 'PTRACCAT_Black', 'PTRACCAT_Hawaiian/Other PI',
       'PTRACCAT_More than one', 'PTRACCAT_White', 'PTMARRY_Divorced',
       'PTMARRY_Married', 'PTMARRY_Never married', 'PTMARRY_Widowed',
       'APOE4_0.0', 'APOE4_1.0', 'APOE4_2.0'],
      dtype='object')

In [241]:
''' Create ADNI2_missing for imputation '''
#ADNI2_remove_categorical = ADNI2[[column for column in ADNI2.columns if column not in categorical_predictors]]
ADNI2_remove_categorical = ADNI2[continuous_predictors]

''' Create the Dataframe for imputation '''
ADNI2_missing = pd.concat([ADNI2['VISCODE'], ADNI2_remove_categorical, ADNI2_categorical], axis=1)

In [242]:
print(ADNI2_remove_categorical.shape)
print(ADNI2_categorical.shape)

(6937, 36)
(6937, 17)


In [243]:
ADNI2_missing.head()

,VISCODE,AGE,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,...,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0
58,bl,66.9,0.0,10.0,15.0,29,42.0,5.0,2.0,20.0,...,0,0,1,0,1,0,0,0,1,0
59,m24,67.5,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,0,0,0,1,0,1,0,0
68,m06,67.5,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,0,0,0,1,0,1,0,0
91,bl,67.5,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0,...,0,0,0,0,0,1,0,1,0,0
92,bl,68.5,0.0,4.0,7.0,30,61.0,6.0,3.0,20.0,...,0,0,1,0,1,0,0,1,0,0


In [244]:
print(ADNI2_missing.shape)

(6937, 54)


In [245]:
ADNI2_missing = ADNI2_missing[ADNI2_missing['VISCODE'] == 'bl']
ADNI2_missing = ADNI2_missing.drop(labels='VISCODE', axis=1)

In [246]:
print(ADNI2_missing.shape)

(789, 53)


In [247]:
full_dataframe =  copy(ADNI2_missing)
missing_rows = []
for i in range(ADNI2_missing.shape[0]):
    if pd.isnull(ADNI2_missing.values[i]).any():
        missing_rows.append(i)
full_dataframe = full_dataframe.drop(full_dataframe.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_missing)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe.shape))

The origin number of data: 789
The number of data that contains missing entries: 288
The dimension of full dataframe: (501, 53)


In [248]:
''' To check that full_dataframe has no missing values '''
np.sum(np.sum(full_dataframe.isnull()))

0

In [249]:
''' Model-Based Imputation - Linear Regression '''

from sklearn.metrics import mean_squared_error

lr = LinearRegression(fit_intercept = True)

for i in range(ADNI2_missing.shape[0]):
    if i % 1000 == 0:
        print(i)
    if ADNI2_missing.iloc[i, :].isnull().any():
        missing_columns = ADNI2_missing.columns[ADNI2_missing.iloc[i, :].isnull()]
        #print(i, missing_columns)
        y_imp = full_dataframe[missing_columns]
        X_imp = full_dataframe.drop(missing_columns, 1)
        lr.fit(X_imp,y_imp)
        X_missing = ADNI2_missing.iloc[i, :].drop(missing_columns)
        y_hat = lr.predict(X_missing)
        ADNI2_missing.loc[ADNI2_missing.index[i], missing_columns] = y_hat[0] 
     

0


In [250]:
ADNI2_missing.shape

(789, 53)

In [251]:
ADNI2_missing.columns[0:3]

Index(['AGE', 'CDRSB_bl', 'ADAS11_bl'], dtype='object')

In [252]:
np.sum(np.sum(ADNI2_missing.isnull()))

0

In [253]:
ADNI2_missing['DX_bl'] = ADNI2['DX_bl'][ADNI2['VISCODE']=='bl']

In [254]:
ADNI2_missing.to_csv(path_or_buf='ADNI2_mb_impute_v3.csv', index=False)

In [255]:
ADNI2_missing.head()

,AGE,CDRSB_bl,ADAS11_bl,ADAS13_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,FAQ_bl,...,PTRACCAT_More than one,PTRACCAT_White,PTMARRY_Divorced,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Widowed,APOE4_0.0,APOE4_1.0,APOE4_2.0,DX_bl
58,66.9,0.0,10.0,15.0,29,42.0,5.0,2.0,20.0000,0.0,...,0,1,0,1,0,0,0,1,0,4
91,67.5,0.0,3.0,5.0,30,42.0,7.0,2.0,20.0000,0.0,...,0,0,0,0,1,0,1,0,0,4
92,68.5,0.0,4.0,7.0,30,61.0,6.0,3.0,20.0000,0.0,...,0,1,0,1,0,0,1,0,0,4
142,80.2,0.0,3.0,10.0,29,36.0,8.0,7.0,63.6364,0.0,...,0,1,0,0,1,0,0,1,0,4
174,71.8,0.5,4.0,6.0,28,44.0,8.0,4.0,40.0000,1.0,...,0,1,0,0,0,1,0,1,0,4


# Below are More Preliminary Exploration by Carlo

In [256]:
## Drop the predictors that have more than 50% missing (thresh parameter)

ADNI2_clean= ADNI2.dropna(axis=1, how='any', thresh=len(ADNI2['DX_bl'])/2, inplace=False)
print("Shape of ADNI2 before dropping columns with NaN>50%")
print(ADNI2.shape)
print("\n")
print("Shape of ADNI2 after dropping columns with NaN>50%")
print(ADNI2_clean.shape)

Shape of ADNI2 before dropping columns with NaN>50%
(6937, 81)


Shape of ADNI2 after dropping columns with NaN>50%
(6937, 81)


In [257]:
ADNI2_clean_remove_all_missing = ADNI2.dropna(axis=1, how='any', thresh=len(ADNI2['DX_bl']), inplace=False)

In [258]:
ADNI2_clean_remove_all_missing.shape

(6937, 19)

In [259]:
np.sum(np.sum(ADNI2_clean_remove_all_missing.isnull()))

0

In [260]:
ADNI2_clean_remove_all_missing.to_json(path_or_buf='ADNI2_clean_remove_all_missing.json' )

In [261]:
with open("ADNI2_clean_remove_all_missing.json") as json_file: 
    ADNI2_clean_remove_all_missing_load = pd.read_json(path_or_buf='ADNI2_clean_remove_all_missing.json')

In [262]:
ADNI2_clean_remove_all_missing_load.head()

,AGE,CDRSB_bl,COLPROT,DX_bl,EXAMDATE,EXAMDATE_bl,M,MMSE_bl,Month,Month_bl,ORIGPROT,PTEDUCAT,PTGENDER,PTID,RID,SITE,VISCODE,Years_bl,update_stamp
10000,55.0,1.0,ADNI2,3,2012-01-26,2011-07-27,6,28,6,6.00000,ADNI2,14,Female,035_S_4114,4114,35,m06,0.501027,2017-10-06 23:19:54.0
10001,55.0,1.0,ADNI2,3,2011-11-02,2011-07-27,3,28,3,3.21311,ADNI2,14,Female,035_S_4114,4114,35,m03,0.268309,2017-08-29 23:07:16.0
10002,55.0,1.0,ADNI2,3,2011-07-27,2011-07-27,0,28,0,0.00000,ADNI2,14,Female,035_S_4114,4114,35,bl,0.000000,2017-10-06 23:19:54.0
10003,70.8,0.0,ADNI2,1,2015-07-29,2011-07-22,48,30,48,48.13110,ADNI2,18,Female,011_S_4105,4105,11,m48,4.019170,2017-10-06 23:19:54.0
10004,70.8,0.0,ADNI2,1,2014-01-16,2011-07-22,30,30,30,29.80330,ADNI2,18,Female,011_S_4105,4105,11,m30,2.488710,2017-08-29 23:07:16.0


In [263]:
# The percentage for -4 and -1 is really small, so in order to make things easier in the imputation, I just delete these
# observations
ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -4].index)
print("Shape of ADNI2 after dropping observations containing -4")
print(ADNI2_clean.shape)
print("\n")

ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -1 ].index)
print("Shape of ADNI2 after dropping observations containing -1")
print(ADNI2_clean.shape)


Shape of ADNI2 after dropping observations containing -4
(6937, 81)


Shape of ADNI2 after dropping observations containing -1
(6937, 81)


In [264]:
#Check the unique values for each predictor. 
for col in ADNI2_clean.columns:
    print('unique values for', col , len(ADNI2_clean[col].value_counts()))

unique values for RID 1184
unique values for PTID 1184
unique values for VISCODE 23
unique values for SITE 58
unique values for COLPROT 1
unique values for ORIGPROT 3
unique values for EXAMDATE 1355
unique values for DX_bl 5
unique values for AGE 293
unique values for PTGENDER 2
unique values for PTEDUCAT 14
unique values for PTETHCAT 2
unique values for PTRACCAT 6
unique values for PTMARRY 4
unique values for APOE4 3
unique values for CDRSB 28
unique values for ADAS11 63
unique values for ADAS13 79
unique values for MMSE 29
unique values for RAVLT_immediate 76
unique values for RAVLT_learning 18
unique values for RAVLT_forgetting 20
unique values for RAVLT_perc_forgetting 91
unique values for FAQ 31
unique values for MOCA 31
unique values for EcogPtMem 57
unique values for EcogPtLang 68
unique values for EcogPtVisspat 51
unique values for EcogPtPlan 29
unique values for EcogPtOrgan 37
unique values for EcogPtDivatt 19
unique values for EcogPtTotal 486
unique values for EcogSPMem 58
un

Hi guys, <br>
I went through all the predictors. It took me a while to go through all the predictors but here below you see their description. <br>
Here below there is my strategy; I will discuss it with Hsiang and we will implement it <br>
** DOUBLE CLICK on it to see it**




                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 

In [265]:
ADNI2_clean.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'DX', 'EXAMDATE_bl',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan

<font color = magenta> 
# In the cell below, it originally had 'FSVERSION' to drop, but I got error message: labels ['FSVERSION'] not contained in axis

In [266]:
#Drop columns
print(ADNI2_clean.shape)
ADNI2_clean.drop (['M','Month', 'Month_bl','update_stamp',
                                 'EXAMDATE','ORIGPROT' ,
                                'COLPROT', 'SITE','ORIGPROT'], axis =1, inplace= True)

(6937, 81)


In [267]:
ADNI2_clean['EcogSPTotal_bl'].unique()

array([ 1.25641,  1.25   ,  1.37838,  1.05128,  2.83333,  1.17949,
        1.     ,  1.12821,      nan,  1.10256,  1.4375 ,  1.02564,
        2.05882,  2.     ,  3.42857,  1.48718,  1.30769,  1.20513,
        3.4359 ,  1.46154,  1.54054,  1.92308,  1.26316,  1.72222,
        1.15385,  2.33333,  3.55263,  1.51282,  3.76923,  1.19444,
        1.61538,  2.64103,  1.76923,  1.29167,  1.18919,  1.02632,
        1.07692,  1.48649,  2.94444,  1.5    ,  3.34211,  1.82051,
        1.83784,  1.71875,  1.4359 ,  1.64865,  1.33333,  1.69231,
        3.05263,  2.89744,  1.78947,  1.39474,  1.23077,  2.15385,
        1.34375,  1.6    ,  1.2    ,  1.38462,  1.41026,  1.63889,
        3.46154,  1.36842,  2.5641 ,  1.65789,  2.46154,  2.72973,
        1.22222,  1.90323,  1.79487,  1.28571,  2.81579,  2.14286,
        1.9    ,  2.5    ,  2.82051,  2.74359,  2.69231,  1.125  ,
        2.44444,  2.31429,  2.82857,  1.84615,  3.35897,  2.64865,
        3.53846,  3.     ,  1.73529,  2.25641,  1.97059,  1.73

'AGE'
'PTEDUCAT'
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'


In [268]:
#Hot one encoding
#categorical_columns_g2cols = ['DX_bl', 'PTGENDER', 'PTETHCAT','PTRACCAT','PTMARRY','APOE4']                                 
#ADNI2_clean = pd.get_dummies(ADNI2_clean, columns=categorical_columns_g2cols, prefix = categorical_columns_g2cols , drop_first=True)

In [269]:
print(ADNI2_clean.columns)

Index(['RID', 'PTID', 'VISCODE', 'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT',
       'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'DX', 'EXAMDATE_bl',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 

In [270]:
#Function to split the data in train and test
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test



<font color = magenta>
# RAS 112917 - examining the drug/medication data

In [271]:
# 112917 - exploring drug effects per reccs from my parents - RAS
ADNI2_ECG = pd.read_csv('data/Medical_History/ADNI2_ECG.csv')
BLSCHECK= pd.read_csv('data/Medical_History/BLSCHECK.csv')
NEUROEXM= pd.read_csv('data/Medical_History/NEUROEXM.csv')
NEUROPATH_03_31_16_DICT = pd.read_csv('data/Medical_History/NEUROPATH_03_31_16_DICT.csv')
NEUROPATH_04_06_17 = pd.read_csv('data/Medical_History/NEUROPATH_04_06_17.csv')
PHYSICAL = pd.read_csv('data/Medical_History/PHYSICAL.csv')
VITALS= pd.read_csv('data/Medical_History/VITALS.csv')
AV45VITALS = pd.read_csv('data/Medical_History/AV45VITALS.csv')
INITHEALTH = pd.read_csv('data/Medical_History/INITHEALTH.csv')
MEDHIST = pd.read_csv('data/Medical_History/MEDHIST.csv')
RECMHIST = pd.read_csv('data/Medical_History/RECMHIST.csv')
BACKMEDS = pd.read_csv('data/Medical_History/BACKMEDS.csv')
RECBLLOG = pd.read_csv('data/Medical_History/RECBLLOG.csv')
AV45FOLLOW = pd.read_csv('data/Medical_History/AV45FOLLOW.csv')
RECCMEDS = pd.read_csv('data/Medical_History/RECCMEDS.csv')
ADVERSE = pd.read_csv('data/Medical_History/ADVERSE.csv')
RECADV = pd.read_csv('data/Medical_History/RECADV.csv')


In [272]:
MEDHIST.head()

,Phase,ID,RID,SITEID,VISCODE,VISCODE2,USERDATE,USERDATE2,EXAMDATE,MHSOURCE,...,MH15BDRUG,MH16SMOK,MH16ASMOK,MH16BSMOK,MH16CSMOK,MH17MALI,MH18SURG,MH19OTHR,MHCOMMEN,update_stamp
0,ADNI1,8,2,107,sc,sc,2005-08-17,NaN,2005-08-17,1,...,NaN,0,NaN,NaN,NaN,0,0,0,-4,2005-08-17 00:00:00.0
1,ADNI1,10,1,10,f,f,2005-08-18,NaN,2005-08-18,1,...,NaN,0,NaN,NaN,NaN,0,0,1,-4,2005-08-18 00:00:00.0
2,ADNI1,12,3,107,sc,sc,2005-08-18,NaN,2005-08-18,1,...,NaN,1,NaN,NaN,NaN,1,1,0,-4,2005-08-18 00:00:00.0
3,ADNI1,14,4,10,sc,sc,2005-08-18,NaN,2005-08-18,1,...,NaN,1,NaN,NaN,NaN,0,1,0,-4,2005-08-18 00:00:00.0
4,ADNI1,16,5,107,sc,sc,2005-08-23,NaN,2005-08-23,1,...,NaN,1,NaN,NaN,NaN,0,1,0,-4,2005-08-23 00:00:00.0


In [273]:
# checking out the number of records for patients in ADNI2 phase, compared with other phases
ADMCPATIENTDRUGCLASSES_20170512 = pd.read_csv('data/ADMCPATIENTDRUGCLASSES_20170512.csv')
print(len(ADMCPATIENTDRUGCLASSES_20170512['Phase']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI2']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNI3']))
print(len(ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase']=='ADNIGO']))

14575
7417
0
967


In [274]:
# making a list of all the different drugs, appending it as a new column to "ADMCPATIENTDRUGCLASSES_20170512"
print(len(ADMCPATIENTDRUGCLASSES_20170512['RID'].unique()))
ADMCPATIENTDRUGCLASSES_20170512['NA'].head()

NA_LIST = []
for i in range(ADMCPATIENTDRUGCLASSES_20170512.shape[0]): 
    try: 
        NA_LIST.append(ADMCPATIENTDRUGCLASSES_20170512['NA'][i].split(','))
    except:
        NA_LIST.append(ADMCPATIENTDRUGCLASSES_20170512['NA'][i])

ADMCPATIENTDRUGCLASSES_20170512['NA_LIST'] = NA_LIST

ADMCPATIENTDRUGCLASSES_20170512[['RID', 'NA_LIST']].head()

2500


,RID,NA_LIST
0,2,[-4]
1,2,"[MECLIZINE, LEVAQUIN]"
2,2,[-4]
3,2,[VITAMIN D]
4,2,"[GLYBURIDE, PANTOPRAZOLE, PRILOSEC]"


In [275]:
ADMCPATIENTDRUGCLASSES_20170512.head()

,RID,VISCODE2,Phase,2-amino-1-phenylethanol derivatives,"ACE inhibitors, plain",Acetaminophen,Adrenergic and dopaminergic agents,Adrenergic Uptake Inhibitors,Aldosterone antagonists,Alpha-adrenoreceptor antagonists,...,Sympathomimetic-like Agent,"Thiazides, plain",Thiazolidinediones,Thyroid Hormone Receptor Agonists,Thyroid Hormone Synthesis Inhibitor,Typical Antipsychotic,Tyrosine,Vasodilator Agents,NA,NA_LIST
0,2,bl,ADNI1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4,[-4]
1,2,m06,ADNI1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MECLIZINE,LEVAQUIN","[MECLIZINE, LEVAQUIN]"
2,2,m102,ADNI2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4,[-4]
3,2,m108,ADNI2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VITAMIN D,[VITAMIN D]
4,2,m120,ADNI2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"GLYBURIDE,PANTOPRAZOLE,PRILOSEC","[GLYBURIDE, PANTOPRAZOLE, PRILOSEC]"


In [277]:
ADNI2_drug = ADMCPATIENTDRUGCLASSES_20170512[ADMCPATIENTDRUGCLASSES_20170512['Phase'] == 'ADNI2']
ADNI2_drug.shape

(7417, 104)

In [278]:
all_drug_list = []
na_list_2 = list(ADMCPATIENTDRUGCLASSES_20170512['NA_LIST'])

for L in na_list_2:
    for I in L:
        print(I) # for some reason i need this print line otherwise i get an error. bizarre. 
        all_drug_list.append(I)
        
myset = set(all_drug_list)
all_drug_list = list(myset)

-4
MECLIZINE
LEVAQUIN
-4
VITAMIN D
GLYBURIDE
PANTOPRAZOLE
PRILOSEC
-4
NAPROXEN
IBUPROFEN
ASPIRIN
MUPIROCIN
MULTI-VITAMIN
-4
DAILY MULTIVITAMIN
-4
-4
-4
GLIPIZIDE XL
ASPIRIN
VITAMIN D
-4
ASPIRIN
-4
-4
NAMENDA
TAPAZOLE
CARDURA
RAZADYNE
RAZADYNE
LIPITOR
CARDURA
TAPAZOLE
MULTIVITAMIN
VITAMIN C
ASPIRIN
IBUPROFEN
ACETAMINOPHEN
DOXAZOSIN
VITAMIN E
CALCUIM PLUS VITAMIN D
GLUCOSAMINE
PROSCAR
NORVASC
ENAPRIL
ATROVASTATIN
PROSAR
ENALAPRIL
LIPITOR
VITAMIN A
-4
VITAMIN E
OMEPRAZOL
ATORVASTATIN
DOXAZOSIN
ENALAPRIL
PIROXICAM
FISH OIL
VITAMIN C
ARICEPT
COSOPT
LIPITOR
ERYTHROMYCIN
PLAVIX
TYLENOL
SIMVASTATIN
ASA
METROCREAM
MILL FLAX SEED OIL
-4
OMEGA 3
-4
BETOPTICS
XALATAN
MVI
VIT C
VIT E
ZINC
ASA
CALCIUM PLUS
GLUCOSAMINE CHONDROTIN
OCUVITE PRESERVISION
-4
MELOXICAM
-4
CURCUMIN
IRON
-4
-4
ATENOLOL/CHLORTHAL 50/25
FOSAMAX
LIPITOR
MIRTAZAPINE
MOBIC
SYNTHROID
ASA
CALTRATE 600 PLUS
CENTRUM SLIVER
GARLIC
ZANTAC
-4
CELEBREX
DESOXIMETASONE OINTMENT
VITAMIN C
NAPROXEN
PAROXETINE
DICYCLOMINE
CITRUCEL
ASA
DIOVAN


-4
-4
-4
-4
LORITAB
-4
VITAMIN D
SAME
COQ10
AVODART
DIOVAN HCT
VITAMIN D
PROSTA-RESPONSE
NIACINAMIDE
IMMUCARE
MULTIVITAMIN
VITAMIN D
SAME
B COMPLEX
MUSHROOM
PROBIOTIC
BONE UP
NIACINAMIDE
SAME
MAGNESIUM
UBIQUINOL
TUMERIC
RESVERATROL
BACOPA
ZINC
COCONUT OIL
ASHWAGANDHA
FLOMAX
FISH OIL
DIOVAN HCT
-4
MULTIVITAMINS
VITAMIN D
NIACINAMIDE
MAGNESIUM
TURMERIC
ZINC
COCONUT OIL
ANTIBIOTIC
RAMIPRIL
-4
DIOVAN HCT
-4
ALPHA LIPOIC ACID
VINPOCETINE
RAMIPRIL
COCONUT OIL
-4
ZINC
N-ACETYLCYSTEINE
RAMIPRIL
DIOVAN HCT
FLOMAX
FOLIC ACID
FISH OIL
MULTI VITAMIN
CALCIUM
-4
-4
-4
HYZAAR
POTASSIUM
-4
CLONIDINE
-4
-4
-4
-4
CLONIDINE
CENTRUM SILVER
VITAMIN D
NORVASC
-4
CLONIDINE
CENTRUM SILVER
VITAMIN D
NORVASC
-4
-4
-4
-4
HYZAAR
NORVASC
POTASSIUM
CENTRUM SILVER
ALEVE
TELITHROMYACIN
METFORMIN
SYNVISC
TYLENOL
DARVOCET
MORPHINE
VANCOMYCIN
MVI WITH IRON
VITAMIN C
VITAMIN E
WARFARIN
ACETAMINOPHEN
DOCUSATE SENNA
LIPITOR
AVANDIA
REMENYL
IBUPROFEN
CORTIZON SHOT
-4
LISINOPRIL/HCTZ
ALEVE
CITALOPRAM
MUCINEX
-4
DONEPEZIL
LIS

QUINAPRIL
CALCIUM
MVI
-4
-4
HCTZ
TOPROL XL
METFORMIN
AMLODIPINE
LYCOPENE
CLORD-CLIDI (CHLORDIAZEPROXIDE + CLIDINIUM)
VITAMIN D
VSL #3
ASPIRIN
LESCOL
DIOVAN
VITAMIN E
MULTIVITAMINS
GLUCOSAMINE WITH CHONDROITIN
CITRACAL + VIT D
CENTRUM SILVER
 MULTIVITAMINS
TYLENOL
-4
-4
MILK OF MAGNESIUM CAPSULE
-4
SYNTHROID
-4
-4
-4
-4
ZOLOFT
ZOLOFT
SYNTHROID
PRILOSEC
FLONASE
ASPIRIN
-4
ALEVE
ASPIRIN
NAPROSYN
ZOLOFT
EFFEXOR
PRILOSEC
-4
IPRATROPIUM
KEFLEX
LUNESTA
FLONASE
ARICEPT
AMBIEN
KEFLEX
NAMENDA
VANCOMYCIN
-4
RAZADYNE ER
LIPITOR
ZESTORETIC
ZANTAC
CENTRUM SILVER
B COMPLEX VITAMIN
AMBIEN
ALLEGRA
QUININE SULFATE
PREDFORTE 1%
ZYMAR
-4
PHOSPHATIDYLSERINE
NIACIN
ZOCOR/SIMVASTATIN
GLIPIZIDE
-4
-4
-4
MULTIVITAMIN
GINKO BILOBA
GLUCOSAMINE
ARICEPT
-4
-4
ARICEPT
SERTRALINE
-4
NAPROXEN
COQ10
MULTIVITAMIN
VIAGRA
CALICUM
INSULIN 70/30
MICRONASE
CLINORIL
BENEMID
COLCHICINE
HYDROCHLOROTHIAZIDE
ISOSORBIDE
LOPRESSOR
DILTIAZEM
TRENTOL
CAPTOPRIL
ZOCOR
GABAPENTIN
ASPIRIN
-4
-4
ARICEPT
-4
ATENOLOL
MULTIVITAMINS
VIT B12


-4
MULTIVITAMIN
ASPIRIN
-4
-4
KONSYL
CENTRUM SILVER
NAMENDA
-4
ARICEPT
ALLEGRA
FLOMAX
TIMOLOL
ASPIRIN
VITAMIN C
METAMUCIL
SINGULAIR
LIPITOR
PAXIL
PREVACID
DYAZIDE
-4
-4
METOPROLOL
QUETIAPINE
LORAZEPAM
NEXIUM
EVISTA
IRON
-4
-4
METOPROLOL
FLUOXETINE
ONE-A-DAY
VITAMIN E
VENLAFAXINE
CLARITIN
BUTALBITAL/APAP/CAFFEINE
FEXOFENADINE
NAPROXEN
PROPRANOLOL
-4
ARICEPT
DAILY MULTIVITAMIN
VENLAFAXINE
LORATADINE
FEXOFENADINE
PROPRANOLOL
AMITRIPTYLINE
-4
EXCEDRIN
CYMBALTA
EVISTA
CALCIUM
VITAMIN D
IBUPROFEN
ANTIBIOTIC
NASACORT
-4
CLONIDINE
-4
POTASSIUM
-4
BUSPIRONE
NAMENDA
PEPCID
NAMENDA
MIRTAZAPINE
TAMOXIFEN
ARICEPT
EFFEXOR
TOPROL XL
-4
ATIVAN
CITALOPRAM
SIMVISTATIN
RANITIDINE
NITROGLYCERIN
NIACIN
ATENOLOL
ASA
MULTIPLE VITAMINS
VITAMIN E
DONEPEZIL
-4
METHYLPHENIDATE
ESTROGEN PATCH
CLARINEX
NASONEX
-4
-4
-4
PRILOSEC
CALTRATE
MAGNESIUM
VIT. E
MULT. VIT
ZYRTEC D
ARICEPT
SINGULAR
DITROPAN XL
LEXAPRO
SPIROLACTONE
NORVASC
EVISTA
LEVOTHYROXINE
FOSOMAX
NAMENDA
ZETIA
MULTIVITAMIN
-4
NAMENDA
-4
AMIODARONE
-4
AR

TRAMADOL
MIRALAX
-4
ASPIRIN
GLUCOSAMINE
LYRICA
ACETAMINOPHEN
NAMENDA
MUCINEX
-4
BETAMETHASONE
TOBRADEX EYE DROPS
VIGAMOX EYE DROPS
OCULAR EYE DROPS
-4
MESALAMINE ENEMA
TRAMADOL
-4
ARICEPT
POTASSIUM CHLORIDE
NORVASC
PREVACID
AZATHIOPRINE
CALCIUM
MESALAMINE
TRAMADOL
NAMENDA
EXELON
-4
FISH OIL
FERROUS SULFATE
FERROUS SULFATE
-4
EFFEXOR
LANSOPRAZOLE
-4
TYLENOL
CHOLESTYRAMINE
HYZAAR
NORVASC
METANX
PREVACID
AZITHROPINE
LYRICA
ASACOL
VITAMIN E
CALCIUM
GLUCOSAMINE
MESALAMINE ENEMA
FOSOMAX
CRESTOR
TOPROL XL
AVALIDE
ASPIRIN
PLAVIX
LEXAPRO
GLUCOSAMINE
VITAMIN C
CENTRUM SILVER- MVI
VITAMIN E
TOPROL XL
LIPITOR
CALCIUM
NICOTINE
-4
-4
GINKO BILOBA PLUS TABLET
EXELON PATCH
PS PILL  (PHOSPHATIDYLSCHOLINE)
VITAMIN E
FOLIC ACID
OMEGA 3
-4
NAMENDA
MULTIVITAMIN
ACTONEL
ARICEPT
LYRICA
NIPHEDAPINE
-4
OMEPRAZOLE
OMEGA-3
-4
LIPITOR
HYDROCODONE - APAP
PANTOPRAZOLE
SIMVASTATIN
LIPITOR
BABY ASPIRIN
MULTIVITAMIN
VITAMIN E
CALCIUM
CHROMIUM PICOLONATE
-4
-4
CALCIUM
-4
COUMADIN
ALTACE
SOTALOL
COUMADIN
-4
-4
-4
ALTACE

COREG
COUMADIN
LEVOXYL
XALATAN
LIPITOR
TYLENOL
MELATONIN
SYSTANE
VITAMIN B12
PLENDIL
VITAMIN D3
-4
MVI
CITRUCEL
FERROUS SULFATE
CALCIUM CARBONATE
ASPIRIN
PLENDIL
MVI
-4
ALBUTEROL
VESICARE
-4
VITAMIN C
MELATONIN
COLACE
VITAMIN D3
KLONOPIN
GABAPENTIN
FERROUS SULFATE
MIRALAX
CIPRO
MELATONIN
COUMADIN
LISINOPRIL
HCTZ
LEVOXYL
FOLIC ACID
TYLENOL
MVI
XALATAN 0.005%
-4
VISION GUARD -VITAMIN FOR EYE HEALTH
PROSTATE VITAMIN
SIMVASTATIN
-4
-4
ARICEPT
NAMENDA
TRIAMTERENE HCTZ
FLOMAX
LEXAPRO
ASPIRIN
MULTIVITAMIN
OMEGA 3 SUPPLEMENT
GINGKO BILOBA
-4
GLUCOSAMINE/CHONDROTIN
PRAVASTATIN
PRILOSEC
COUMADIN
AUGMENTIN
GLUCOSAMINE/CHONDROTIN
LISINOPRIL/HCTZ
-4
GLUCOSAMINE/CHONDROTIN
MULTIVITAMIN
NASAREL
FISH OIL
APIRIN
DOXAZOSIN MESYLATE
AMLODIPINE
SIMVASTATIN
-4
FLUTICASONE
AMLODIPINE
DOXAZOSIN
LISINOPRIL
ASPIRIN
GLUCOSAMINE
MVI
NORWEGIAN SALMON OIL
SIMVASTATIN
-4
FLUTICASONE
AMLODIPINE
DOXAZOSIN
LISINOPRIL
ASPIRIN
SIMVASTATIN
GLUCOSAMINE
MVI
-4
-4
-4
AMLODIPINE
MUPIROCIN
FUROSEMIDE
POTASSIUM CHLORIDE
PRAVAS

HYDROCODONE
-4
-4
LUPRON
-4
MULTIVITAMIN
LUPRON
CALCIUM +D
SOY EXTRACT
FLAX SEED
GLUCOSAMINE/CHONDROITEN
BABY ASPIRIN
VITAMIN E
FOLIC ACID
ZYRTEC
CALCIUM +D
FLAXSEED OIL
GLUCOSAMINE
-4
BABY ASPIRIN
VITAMIN E
FOLIC ACID
VITAMIN C
SYNTHROID
-4
SYNTHROID
-4
-4
ASPIRIN
ZYRTEC
ARICEPT
TORSEMIDE
MULTI VITAMIN
ASPIRIN
PROTONIX
-4
FLOMAX
-4
-4
-4
LUTINE
LAMISIL
VOLTAREN GEL
-4
LEVOTHYROXINE
MELOXICAM
MAGNESIUM
ZANTAC
ASPIRIN
DAILY MULTIVITAMIN
FISH OIL
GLUCOSAMINE
FOLIC ACID
CALCIUM
ZYRTEC
-4
VOLTAREN
MELOXICAM
MAGNESIUM
LEVOTHYROXINE
-4
-4
-4
-4
VITAMIN D3
-4
ZANTAC
VITAMIN E
ASPIRIN
MULTIVITAMIN
FISH OIL
GLUCOSAMINE
ZINC
FOLIC ACID
SAW PALMETTO
SELENIUM
CALCIUM
VITAMIN C
IBUPROFEN
ZYRTEC
-4
-4
RAMIPRIL
EZETIMIBE
VITAMIN B12
LIPITOR
ASPIRIN
EXTRA STRENGTH TYLENOL
SYNTHROID
ASPIRIN
AMITRIPTYLINE
-4
RAMIPRIL
ALENDRONATE
-4
-4
VITAMIN D
SELENIUM
VITAMIN E
MULTIVITAMINS
VITALUX
GLUCOSAMINE & CHONDROITIN & MSM
FLOMAX
LIPITOR
ASA
TYLENOL EXTRA STRENGTH
AMITRIPTYLINE
RAMIPRIL
ALENDRONATE
VITAMIN D
O

EVISTA
POTASSIUM
PROTONIX
CIPRO
GALANTAMINE ER
ATORVASTATIN CALCIUM
QUINAPRIL HYDROCHLORIDE
ENTERIC COATED ACETYLSALICYLIC ACID
VITAMIN E
VITAMIN C
MULTIVITAMINS + MINERALS
-4
-4
-4
-4
FAST WORK FOR JOINTS
B COMPLEX
-4
MULTI VITAMIN
VITAMIN E
VITAMIN C
ASPIRIN
AVODART
UROXATRAL
SAW PALMETTO
MULTIVITAMIN
VITAMIN E
GLUCOSAMINE AND CHONDROITIN
CALCIUM
ASPIRIN
LIPITOR
-4
-4
-4
NAMENDA
RAZADYNE
WELLBUTRIN
VASOTEC
HCTZ
COGNITEX WITH PREGNOLONE
NEWU (MUSCADINE GRAPES)
FISH OIL
ASPIRIN
VIT E
VIT C
FOLIC ACID
CALCIUM PLUS VIT D
MULTIVITAMIN
MAGNESIUM
-4
-4
VITAMIN B-12
METRONIDAZOLE
CETIRIZINE
FOLTX
LEVOTHYROXINE
FERROUS SULFATE
-4
AMOXICILLIN
ECOTRIN
MULTI VITAMIN
LIPITOR
ZETIA
DIOVAN
METRONIDAZOLE
CETIRIZINE
LEVOTHYROXINE
TYLENOL ARTHRITIS
OSTEO BIFLEX
-4
ZYRTEC
ECOTRIN
MULTI VITAMIN
LIPITOR
ZETIA
FOLTX
FERROUS SULPHATE
DIOVAN


TypeError: 'float' object is not iterable

In [280]:
all_drug_list = set(all_drug_list)

In [295]:
# find out which names are in the database

# blood thinning drugs
print('Prescription blood thinners:')
print('DABIGATRAN' in all_drug_list)
print('PRADAXA' in all_drug_list)
print('RIVAROXABAN' in all_drug_list)
print('XARELTO' in all_drug_list)
print('APIXABAN' in all_drug_list)
print('ELIQUIS' in all_drug_list)
print('HEPARIN' in all_drug_list)
print('WARFARIN' in all_drug_list)
print('COUMADIN' in all_drug_list)

# OTC blood thinners
print('OTC blood thinners:')
print('ASPIRIN' in all_drug_list)

# calcium
print('CALCIUM' in all_drug_list)
print('CA' in all_drug_list)
print('CA2+' in all_drug_list)

# vitamin D
print('VITAMIN D' in all_drug_list)
print('VIT D' in all_drug_list)
print('VITA D' in all_drug_list)

Prescription blood thinners:
True
False
False
False
False
False
True
True
True
OTC blood thinners:
True
True
True
False
True
True
False


In [313]:
list_of_drugs_in_data_maybe = ['DABIGATRAN','PRADAXA','RIVAROXABAN','XARELTO','APIXABAN','ELIQUIS','HEPARIN','WARFARIN',
                         'COUMADIN', 'ASPIRIN','CALCIUM','CA', 'CA2+','VITAMIN D','VIT D','VITA D']
list_of_drugs_in_data = []

for i in range(len(list_of_drugs_in_data_maybe)):
    if list_of_drugs_in_data_maybe[i] in all_drug_list:
        list_of_drugs_in_data.append(list_of_drugs_in_data_maybe[i])
list_of_drugs_in_data       

['DABIGATRAN',
 'HEPARIN',
 'WARFARIN',
 'COUMADIN',
 'ASPIRIN',
 'CALCIUM',
 'CA',
 'VITAMIN D',
 'VIT D']